# Migrate Views Between Catalogs
This notebook copies all views from a source catalog to a destination catalog. Provide the catalog names below and run all cells.

In [ ]:

%md
## Configure Source and Destination Catalogs

In [ ]:

try:
    dbutils.widgets.text("1.source_catalog", "source_catalog")
    dbutils.widgets.text("2.destination_catalog", "destination_catalog")
    source_catalog = dbutils.widgets.get("1.source_catalog")
    destination_catalog = dbutils.widgets.get("2.destination_catalog")
except NameError:
    # When running as a script
    source_catalog = "source_catalog"
    destination_catalog = "destination_catalog"


## Migrate Views

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.getActiveSession() or SparkSession.builder.getOrCreate()


def get_views(catalog, view_type):
    query = f"""
    SELECT table_schema, table_name, view_definition
    FROM `{catalog}`.information_schema.{view_type}
    WHERE table_schema NOT IN ('information_schema')
    """
    return spark.sql(query)


views = get_views(source_catalog, "views")


def recreate_views(df, view_kind):
    for row in df.collect():
        schema = row["table_schema"]
        name = row["table_name"]
        definition = row["view_definition"].rstrip(";")
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{destination_catalog}`.`{schema}`")
        spark.sql(f"CREATE OR REPLACE {view_kind} `{destination_catalog}`.`{schema}`.`{name}` AS {definition}")
        print(f"Created {view_kind} {destination_catalog}.{schema}.{name}")


recreate_views(views, "VIEW")
